In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('casewhen').master('local[1]').getOrCreate()

In [2]:
df=spark.read.csv('C:\\Users\\Raju\\Documents\\Survey-2020.csv')
df.printSchema()
df.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)

+----+--------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+--------------------+
| _c0|                 _c1|                 _c2|                 _c3|               _c4|          _c5|                 _c6|                 _c7|      _c8|                 _c9|
+----+--------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+--------------------+
|Year|Industry_aggregat...|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variabl

In [3]:
df1=spark.read.format('csv').option('header','true').load('C:\\Users\\Raju\\Documents\\Survey-2020.csv')
df1.printSchema()
df1.show(truncate=0)
print(type(df1))

root
 |-- Year: string (nullable = true)
 |-- Industry_aggregation_NZSIOC: string (nullable = true)
 |-- Industry_code_NZSIOC: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Variable_code: string (nullable = true)
 |-- Variable_name: string (nullable = true)
 |-- Variable_category: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Industry_code_ANZSIC06: string (nullable = true)

+----+---------------------------+--------------------+--------------------+------------------+-------------+-----------------------------------------------------+---------------------+---------+----------------------------------------------------------------------------------------------------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|Units             |Variable_code|Variable_name                                        |Variable_category    |Value    |Industry_code_ANZSIC06  

In [4]:
#file formats
print('''text(csv,pipe delimited,tab delimited)
avro
parquet-columnar storage(compression)
orc-optimised row columnar format(update/delete)
json''')

text(csv,pipe delimited,tab delimited)
avro
parquet-columnar storage(compression)
orc-optimised row columnar format(update/delete)
json


In [5]:
from pyspark.sql.functions import col
dfselect=df1.select(col('year'),col('variable_code'),col('Industry_name_NZSIOC'),col('variable_category'))
dfselect.show(truncate=0)

+----+-------------+--------------------+---------------------+
|year|variable_code|Industry_name_NZSIOC|variable_category    |
+----+-------------+--------------------+---------------------+
|2020|H01          |All industries      |Financial performance|
|2020|H04          |All industries      |Financial performance|
|2020|H05          |All industries      |Financial performance|
|2020|H07          |All industries      |Financial performance|
|2020|H08          |All industries      |Financial performance|
|2020|H09          |All industries      |Financial performance|
|2020|H10          |All industries      |Financial performance|
|2020|H11          |All industries      |Financial performance|
|2020|H12          |All industries      |Financial performance|
|2020|H13          |All industries      |Financial performance|
|2020|H14          |All industries      |Financial performance|
|2020|H19          |All industries      |Financial performance|
|2020|H20          |All industries      

In [6]:
dfVarCategories=dfselect.select(col('variable_category')).distinct()
dfVarCategories.show(truncate=0)

+---------------------+
|variable_category    |
+---------------------+
|Financial performance|
|Financial position   |
|Financial ratios     |
+---------------------+



In [7]:
dfselect.printSchema()

root
 |-- year: string (nullable = true)
 |-- variable_code: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- variable_category: string (nullable = true)



In [8]:
dfcast=dfselect.withColumn('Value',col('Value').cast('Integer'))

NameError: name 'dfSelect' is not defined

In [ ]:
dfcast.groupBy(col('Variable_category')).sum('Value') \
     .select(col('Variable_category'),col('sum(Value)').alias('Total_Value')).show(truncate=0)

In [24]:
df_json=spark.read.json('C:\\Users\\Raju\\Desktop\\sample.json',multiLine=True)
df_json.show(truncate=0)
d=df_json.drop('gender')

+--------------------+---+---------+------+--------+--------------------+
|address             |age|firstName|gender|lastName|phoneNumbers        |
+--------------------+---+---------+------+--------+--------------------+
|{San Diego, CA, 101}|28 |Joe      |male  |Jackson |[{7349282382, home}]|
+--------------------+---+---------+------+--------+--------------------+



In [49]:
df=d.drop('age','lastName','address')
df.show(truncate=0)

+---------+--------------------+
|firstName|phoneNumbers        |
+---------+--------------------+
|Joe      |[{7349282382, home}]|
+---------+--------------------+



In [30]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- streetAddress: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- phoneNumbers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [51]:
from pyspark.sql.functions import explode
df_f=df.withColumn('phoneNumbers_flat',explode('phoneNumbers'))
df_f=df_f.drop('phoneNumbers')

In [52]:
df_f.show()

+---------+------------------+
|firstName| phoneNumbers_flat|
+---------+------------------+
|      Joe|{7349282382, home}|
+---------+------------------+

